In [139]:
import pandas as pd, sqlite3

# Ex03. Aggregations

## 1. Подключение к базе данных

In [131]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
cursor = conn.cursor()

## 2. Получение данных о таблице

In [132]:
pd.read_sql('PRAGMA table_info(test)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [133]:
cursor.execute('SELECT * FROM test LIMIT 10')
pd.DataFrame(cursor.fetchall())

,0,1,2,3,4
0,0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## 3. Минимальное значение дельты между первым коммитом и дедлайном

In [134]:
cursor.execute('''
               SELECT t.uid, CAST(MIN((JULIANDAY(t.first_commit_ts) - JULIANDAY(DATETIME(d.deadlines, 'unixepoch'))) * 24) AS INTEGER)
               FROM test t
               JOIN deadlines d ON d.labs = t.labname AND t.labname != 'project1'
               ''')
df_min = pd.DataFrame(cursor.fetchall(), columns=['uid', 'diff'])
df_min

,uid,diff
0,user_30,-202


## 4. Максимальное значение дельты между первым коммитом и дедлайном

In [135]:
cursor.execute('''
               SELECT t.uid, CAST(MAX((JULIANDAY(t.first_commit_ts) - JULIANDAY(DATETIME(d.deadlines, 'unixepoch'))) * 24) AS INTEGER)
               FROM test t
               JOIN deadlines d ON d.labs = t.labname AND t.labname != 'project1'
               ''')
df_max = pd.DataFrame(cursor.fetchall(), columns=['uid', 'diff'])
df_max

,uid,diff
0,user_25,-2


## 5. Среднее значение дельты между первым коммитом и дедлайном

In [136]:
cursor.execute('''
               SELECT AVG(CAST(((JULIANDAY(t.first_commit_ts) - JULIANDAY(DATETIME(d.deadlines, 'unixepoch'))) * 24) AS INTEGER))
               FROM test t
               JOIN deadlines d ON d.labs = t.labname AND t.labname != 'project1'
               ''')
df_avg = pd.DataFrame(cursor.fetchall(), columns=['avg'])
df_avg

,avg
0,-89.125


## 6. Матрица корреляции

*Мы хотим проверить гипотезу о том, что пользователи, посетившие ленту новостей всего несколько раз, имеют меньшую разницу между первым коммитом и дедлайном*

In [137]:
cursor.execute('''
               SELECT t.uid, CAST(((JULIANDAY(t.first_commit_ts) - JULIANDAY(DATETIME(d.deadlines, 'unixepoch'))) * 24) AS INTEGER), COUNT(DISTINCT(p.datetime))
               FROM test t
               JOIN deadlines d ON d.labs = t.labname AND d.labs != 'project1'
               JOIN pageviews p ON p.uid = t.uid
               GROUP BY t.uid''')
views_diff = pd.DataFrame(cursor.fetchall(), columns=['uid', 'diff', 'pageviews'])
views_diff.corr(numeric_only=True)

,diff,pageviews
diff,1.000000,-0.062967
pageviews,-0.062967,1.000000


## 7. Закрытие соединения

In [138]:
conn.close()